In [1]:
# %run MyFunctions.ipynb
# %load_ext autoreload
# %autoreload 2
# import import_ipynb
# import MyFunctions
# from MyFunctions import *

import datetime
today = datetime.datetime.today().strftime('%m-%d-%Y')
import smtplib
import simplejson
import pymssql
import psycopg2
import pandas as pd
import datetime
from pandasql import sqldf

In [50]:
df = generate_master_file_fy20()

In [27]:
generate_fills_report(df2, today)

In [28]:
generate_pending_offers_report(df2, today)

In [31]:
generate_all_offers_report(df2, today)

In [29]:
generate_opens_report(df2, today)

In [10]:
df_pdd_v2 = generate_pdd_df_v2()

In [9]:
generate_open_funnel_report(df_pdd_v2, today)

In [2]:
def generate_master_file_fy20():
    
    # generate credentials for gh, pdd; queries for gh, pdd; columns in dataframe
    with open("/Users/maxwell.lee/OneDrive - Jet/New Folder/Notebooks/Credentials/redshift_creds.json.nogit") as fh:
        creds_gh = simplejson.loads(fh.read())
        
    with open("/Users/maxwell.lee/OneDrive - Jet/New Folder/Notebooks/Credentials/sqlserver_creds.json.nogit") as fh:
        creds_pdd = simplejson.loads(fh.read())    
    
    file = open('/Users/maxwell.lee/OneDrive - Jet/New Folder/Queries/candidate_date_details_all.sql', 'r')
    sql_gh = file.read()
    
    file = open('/Users/maxwell.lee/OneDrive - Jet/New Folder/Queries/candidate_date_details_kenexa_v2.sql', 'r')
    sql_pdd = file.read()
    
    with open('/Users/maxwell.lee/OneDrive - Jet/New Folder/Queries/master_file_columns.txt') as f:
        columns = f.read().splitlines()
        
    with open('/Users/maxwell.lee/OneDrive - Jet/New Folder/Queries/date_columns.txt') as f:
        date_columns = f.read().splitlines()
        
    with open('/Users/maxwell.lee/OneDrive - Jet/New Folder/Queries/time_columns.txt') as f:
        time_columns = f.read().splitlines()          
        
    # connect to greenhouse
    conn_red = psycopg2.connect(host = creds_gh['host_name'], 
                                port = creds_gh['port_num'], 
                                database = creds_gh['db_name'], 
                                user = creds_gh['user_name'],
                                password = creds_gh['password'])
    
    # open cursor, run the query, fetch results, close cursor, close connection, save results to dataframe
    cur = conn_red.cursor()
    cur.execute(sql_gh)
    results = cur.fetchall()
    cur.close()
    conn_red.close()
    df_gh = pd.DataFrame(results)
    
    # connect to pdd
    conn_mssql = pymssql.connect(server = creds_pdd['server'],
                                 user = creds_pdd['user_name'],
                                 password = creds_pdd['password'])
    
    # open cursor, run the query, fetch results, close cursor, close connection, save results to dataframe
    cur = conn_mssql.cursor()
    cur.execute(sql_pdd)
    results = cur.fetchall()
    cur.close()
    conn_mssql.close()
    df_pdd = pd.DataFrame(results)
    
    # save and merge frames, convert dates to  and write to excel file
    frames = [df_gh, df_pdd]
    df = pd.concat(frames)
    df.columns = columns
    df = df.reset_index(drop = True)
    # df.to_excel('testfile.xlsx', index = False)   
    
#     df[date_columns] = df[date_columns].apply(pd.to_datetime)
    
#     for col in date_columns:
#         df[col] = df[col].apply(lambda x: x.strftime('%m-%d-%Y'))

    for col in date_columns:
        df[col] = pd.to_datetime(df[col]).apply(lambda x: x.strftime('%m-%d-%Y') if not pd.isnull(x) else '')
    
    return df
        

In [3]:
def generate_fills_report(dataframe, file_date):
    
    # create new df, drop rows and write to excel
    
    # these are the columns used in this report
    with open('/Users/maxwell.lee/OneDrive - Jet/New Folder/Queries/fills_report_columns.txt') as f:
        columns = f.read().splitlines()
    
    df = dataframe
    # drop all candidates with no offer_accepted_date
    df = df[
            (df['anticipated_start_date'].apply(pd.to_datetime) >= datetime.datetime.strptime('2019-02-01', '%Y-%m-%d'))
            &
            (df['anticipated_start_date'].apply(pd.to_datetime) < datetime.datetime.strptime('2020-02-01', '%Y-%m-%d'))
            &
            (df['offer_accepted_date'] != '')
            &
            (df['offer_declined_date'] == '')
            & 
            (df['rejected_date'] == '')
           ]
    
    # for fy19
    # df1 = df[
    #          (df['offer_accepted_date'].apply(pd.to_datetime) >= datetime.datetime.strptime('2018-02-01', '%Y-%m-%d'))
    #          &
    #          (df['offer_accepted_date'].apply(pd.to_datetime) < datetime.datetime.strptime('2019-02-01', '%Y-%m-%d'))
    #         ]
    
    # df1 = df1[columns]
    # df1.to_excel('fills_report_fy19_%s.xlsx' % file_date, index = False)   
    
    # for fy20
    # df2 = df[
    #         (df['offer_accepted_date'].apply(pd.to_datetime) >= datetime.datetime.strptime('2019-02-01', '%Y-%m-%d'))
    #         &
    #         (df['offer_accepted_date'].apply(pd.to_datetime) < datetime.datetime.strptime('2020-02-01', '%Y-%m-%d'))
    #        ]
    
    df = df[columns]
    df.to_excel('fills_report_as_of_%s.xlsx' % file_date, index = False)  

In [4]:
def generate_fills_report_v3(dataframe):
    
    # create new df, drop rows and write to excel
    
    # these are the columns used in this report
    with open('/Users/maxwell.lee/OneDrive - Jet/New Folder/Queries/fills_report_columns.txt') as f:
        columns = f.read().splitlines()
    
    df = dataframe
    # drop all candidates with no offer_accepted_date
    df = df[
            (df['anticipated_start_date'].apply(pd.to_datetime) >= datetime.datetime.strptime('2019-02-01', '%Y-%m-%d'))
            &
            (df['anticipated_start_date'].apply(pd.to_datetime) < datetime.datetime.strptime('2020-02-01', '%Y-%m-%d'))
            &
            (df['offer_accepted_date'] != '')
            &
            (df['offer_declined_date'] == '')
            & 
            (df['rejected_date'] == '')
           ]
    
    # for fy19
    # df1 = df[
    #          (df['offer_accepted_date'].apply(pd.to_datetime) >= datetime.datetime.strptime('2018-02-01', '%Y-%m-%d'))
    #          &
    #          (df['offer_accepted_date'].apply(pd.to_datetime) < datetime.datetime.strptime('2019-02-01', '%Y-%m-%d'))
    #         ]
    
    # df1 = df1[columns]
    # df1.to_excel('fills_report_fy19_%s.xlsx' % file_date, index = False)   
    
    # for fy20
    # df2 = df[
    #         (df['offer_accepted_date'].apply(pd.to_datetime) >= datetime.datetime.strptime('2019-02-01', '%Y-%m-%d'))
    #         &
    #         (df['offer_accepted_date'].apply(pd.to_datetime) < datetime.datetime.strptime('2020-02-01', '%Y-%m-%d'))
    #        ]
    
    df = df[columns]
    df.to_excel('fills_report_test.xlsx', index = False)  

In [23]:
generate_fills_report_v3(df2)

In [5]:
def generate_pending_offers_report(dataframe, file_date):
    
    # create new df, drop rows and write to excel
    
    # these are the columns used in this report
    with open('/Users/maxwell.lee/OneDrive - Jet/New Folder/Queries/fills_report_columns.txt') as f:
        columns = f.read().splitlines()
    
    df = dataframe
    # drop all candidates with null offer_extended_date
    df = df[
            (df['offer_extended_date'].apply(pd.to_datetime) >= datetime.datetime.strptime('2019-02-01', '%Y-%m-%d'))
            &
            (df['offer_extended_date'].apply(pd.to_datetime) < datetime.datetime.strptime('2020-02-01', '%Y-%m-%d'))
            &
            (df['offer_accepted_date'] == '')
            &
            (df['rejected_date'] == '')
            &
            (df['offer_declined_date'] == '')
            &
            (
                (df['ta_current_status_mapped'] == 'Offer Extended') | 
                (df['ta_current_status_mapped'] == 'Hired') | 
                (df['ta_current_status_mapped'] == 'Offer Accepted') |
                (df['ta_current_status_mapped'] == 'Selected')
            )
            &
            ((df['job_status_mapped'] == 'Open') | (df['job_status_mapped'] == 'Hold'))
           ]

    df = df[columns]
    df.to_excel('pending_offers_report_as_of_%s.xlsx' % file_date, index = False)   

In [6]:
def generate_all_offers_report(dataframe, file_date):
    
    # create new df, drop rows and write to excel
    
    # these are the columns used in this report
    with open('/Users/maxwell.lee/OneDrive - Jet/New Folder/Queries/all_offers_report_columns.txt') as f:
        columns = f.read().splitlines()
    
    df = dataframe
    # drop all candidates with null offer_extended_date
    df = df[
            (df['offer_extended_date'].apply(pd.to_datetime) >= datetime.datetime.strptime('2019-02-01', '%Y-%m-%d'))
            &
            (df['offer_extended_date'].apply(pd.to_datetime) < datetime.datetime.strptime('2020-02-01', '%Y-%m-%d'))
           ]

    df = df[columns]
    df.to_excel('all_offers_report_as_of_%s.xlsx' % file_date, index = False)   

In [7]:
def generate_opens_report(dataframe, file_date):
    
    # create new df, drop rows and write to excel
    
    # these are the columns used in this report
    with open('/Users/maxwell.lee/OneDrive - Jet/New Folder/Queries/opens_report_columns.txt') as f:
        columns = f.read().splitlines()
        
    file = open('/Users/maxwell.lee/OneDrive - Jet/New Folder/Queries/opens_report.sql', 'r')
    sql_opens_report = file.read()   
        
    sql_df = dataframe[columns]
    
    out = sqldf(sql_opens_report)
    
    out.to_excel('opens_report_as_of_%s.xlsx' % file_date, index = False)   

In [8]:
def generate_open_funnel_report(dataframe, file_date):
    
    # import columns and sql query for report
    with open('/Users/maxwell.lee/OneDrive - Jet/New Folder/Queries/open_funnel_report_columns.txt') as f:
        columns = f.read().splitlines()
        
    with open('/Users/maxwell.lee/OneDrive - Jet/New Folder/Queries/time_columns.txt') as f:
        time_columns = f.read().splitlines()
        
    file = open('/Users/maxwell.lee/OneDrive - Jet/New Folder/Queries/open_funnels.sql', 'r')
    sql_opens_report = file.read()
    
    # keep columns for report and handle data type issues
    # sql_df = dataframe[columns]
    
    for cols in time_columns:
        dataframe.loc[dataframe[cols].isnull(), cols] = -1
        dataframe.loc[:, cols] = dataframe[cols].apply(lambda x: float(x))
    
    # generate report and write to excel
    out = sqldf(sql_opens_report)
    out.to_excel('cc_open_funnels_report_as_of_%s.xlsx' % file_date, index = False)

In [9]:
def generate_gh_df():
    
    # generate gh credentials
    with open("/Users/maxwell.lee/OneDrive - Jet/New Folder/Notebooks/Credentials/redshift_creds.json.nogit") as fh:
        creds_gh = simplejson.loads(fh.read())
        
    # generate gh query
    file = open('/Users/maxwell.lee/OneDrive - Jet/New Folder/Queries/candidate_date_details_all_v2.sql', 'r')
    sql_gh = file.read()
    
    #generate column lists
    with open('/Users/maxwell.lee/OneDrive - Jet/New Folder/Queries/master_file_columns.txt') as f:
        columns = f.read().splitlines()
        
    with open('/Users/maxwell.lee/OneDrive - Jet/New Folder/Queries/date_columns.txt') as f:
        date_columns = f.read().splitlines()
        
    with open('/Users/maxwell.lee/OneDrive - Jet/New Folder/Queries/time_columns.txt') as f:
        time_columns = f.read().splitlines()      
        
    # connect to greenhouse
    conn_red = psycopg2.connect(host = creds_gh['host_name'], 
                                port = creds_gh['port_num'], 
                                database = creds_gh['db_name'], 
                                user = creds_gh['user_name'],
                                password = creds_gh['password'])
    
    # open cursor, run the query, fetch results, close cursor, close connection, save results to dataframe
    cur = conn_red.cursor()
    cur.execute(sql_gh)
    results = cur.fetchall()
    cur.close()
    conn_red.close()
    df_gh = pd.DataFrame(results)
    df_gh.columns = columns
    
    # convert datatypes
    for col in date_columns:
        df_gh[col] = pd.to_datetime(df_gh[col]).apply(lambda x: x.strftime('%m-%d-%Y') if not pd.isnull(x) else '')
    
    return df_gh

In [10]:
def generate_gh_df_v2():
    
    # generate gh credentials
    with open("/Users/maxwell.lee/OneDrive - Jet/New Folder/Notebooks/Credentials/redshift_creds.json.nogit") as fh:
        creds_gh = simplejson.loads(fh.read())
        
    # generate gh query
    file = open('/Users/maxwell.lee/OneDrive - Jet/New Folder/Queries/candidate_date_details_all_v2.sql', 'r')
    sql_gh = file.read()
    
    #generate column lists
    with open('/Users/maxwell.lee/OneDrive - Jet/New Folder/Queries/master_file_columns.txt') as f:
        columns = f.read().splitlines()
        
    with open('/Users/maxwell.lee/OneDrive - Jet/New Folder/Queries/date_columns.txt') as f:
        date_columns = f.read().splitlines()
        
    with open('/Users/maxwell.lee/OneDrive - Jet/New Folder/Queries/time_columns.txt') as f:
        time_columns = f.read().splitlines()      
        
    # connect to greenhouse
    conn_red = psycopg2.connect(host = creds_gh['host_name'], 
                                port = creds_gh['port_num'], 
                                database = creds_gh['db_name'], 
                                user = creds_gh['user_name'],
                                password = creds_gh['password'])
    
    # open cursor, run the query, fetch results, close cursor, close connection, save results to dataframe
    cur = conn_red.cursor()
    cur.execute(sql_gh)
    results = cur.fetchall()
    cur.close()
    conn_red.close()
    df_gh = pd.DataFrame(results)
    df_gh.columns = columns
    
    # convert datatypes
    for cols in date_columns:
        df_gh[cols] = pd.to_datetime(df_gh[cols], errors = 'coerce').dt.strftime('%m-%d-%Y').replace('NaT', '')
    
    return df_gh

In [11]:
def generate_pdd_df():
    # generate gh credentials
    with open("/Users/maxwell.lee/OneDrive - Jet/New Folder/Notebooks/Credentials/sqlserver_creds.json.nogit") as fh:
        creds_pdd = simplejson.loads(fh.read())    
        
    # generate gh query
    file = open('/Users/maxwell.lee/OneDrive - Jet/New Folder/Queries/candidate_date_details_kenexa_v2.sql', 'r')
    sql_pdd = file.read()
    
    #generate column lists
    with open('/Users/maxwell.lee/OneDrive - Jet/New Folder/Queries/master_file_columns.txt') as f:
        columns = f.read().splitlines()
        
    with open('/Users/maxwell.lee/OneDrive - Jet/New Folder/Queries/date_columns.txt') as f:
        date_columns = f.read().splitlines()
        
    with open('/Users/maxwell.lee/OneDrive - Jet/New Folder/Queries/time_columns.txt') as f:
        time_columns = f.read().splitlines()
        
    # connect to pdd
    conn_mssql = pymssql.connect(server = creds_pdd['server'],
                                 user = creds_pdd['user_name'],
                                 password = creds_pdd['password'])
    
    # open cursor, run the query, fetch results, close cursor, close connection, save results to dataframe
    cur = conn_mssql.cursor()
    cur.execute(sql_pdd)
    results = cur.fetchall()
    cur.close()
    conn_mssql.close()
    df_pdd = pd.DataFrame(results)     
    df_pdd.columns = columns

    # convert datatypes
    for col in date_columns:
        df_pdd[col] = pd.to_datetime(df_pdd[col]).apply(lambda x: x.strftime('%m-%d-%Y') if not pd.isnull(x) else '')
    
    return df_pdd    

In [12]:
def generate_pdd_df_v2():
    # generate gh credentials
    with open("/Users/maxwell.lee/OneDrive - Jet/New Folder/Notebooks/Credentials/sqlserver_creds.json.nogit") as fh:
        creds_pdd = simplejson.loads(fh.read())    
        
    # generate gh query
    file = open('/Users/maxwell.lee/OneDrive - Jet/New Folder/Queries/candidate_date_details_kenexa_v2.sql', 'r')
    sql_pdd = file.read()
    
    #generate column lists
    with open('/Users/maxwell.lee/OneDrive - Jet/New Folder/Queries/master_file_columns.txt') as f:
        columns = f.read().splitlines()
        
    with open('/Users/maxwell.lee/OneDrive - Jet/New Folder/Queries/date_columns.txt') as f:
        date_columns = f.read().splitlines()
        
    with open('/Users/maxwell.lee/OneDrive - Jet/New Folder/Queries/time_columns.txt') as f:
        time_columns = f.read().splitlines()
        
    # connect to pdd
    conn_mssql = pymssql.connect(server = creds_pdd['server'],
                                 user = creds_pdd['user_name'],
                                 password = creds_pdd['password'])
    
    # open cursor, run the query, fetch results, close cursor, close connection, save results to dataframe
    cur = conn_mssql.cursor()
    cur.execute(sql_pdd)
    results = cur.fetchall()
    cur.close()
    conn_mssql.close()
    df_pdd = pd.DataFrame(results)     
    df_pdd.columns = columns

    # convert datatypes
    for cols in date_columns:
        df_pdd[cols] = pd.to_datetime(df_pdd[cols], errors = 'coerce').dt.strftime('%m-%d-%Y').replace('NaT', '')
    
    return df_pdd    

In [13]:
def merge_dataframes(df1, df2):
    frames = [df1, df2]
    df = pd.concat(frames)
    df = df.reset_index(drop = True)
    
    return df

In [17]:
df_gh = generate_gh_df()

In [ ]:
df_pdd = generate_pdd_df()

In [14]:
df_gh_v2 = generate_gh_df_v2()

In [ ]:
df_pdd_v2 = generate_pdd_df_v2()

In [22]:
df2 = merge_dataframes(df_gh_v2, df_pdd_v2)

In [2]:
import datetime as dt

In [70]:
def to_datetime_test(df):
    with open('/Users/maxwell.lee/OneDrive - Jet/New Folder/Queries/date_columns.txt') as f:
        date_columns = f.read().splitlines()
        
    for cols in date_columns:
        df[cols] = pd.to_datetime(df[cols], errors = 'coerce').dt.strftime('%m-%d-%Y').replace('NaT', '')
        
    return df    

def to_astype_test(df):
    with open('/Users/maxwell.lee/OneDrive - Jet/New Folder/Queries/date_columns.txt') as f:
        date_columns = f.read().splitlines()
        
    for cols in date_columns:
        df[cols] = df[cols].astype('datetime64[D]').dt.strftime('%m-%d-%Y').replace('NaT', '')
        
    return df            

In [75]:
df_dt = to_datetime_test(df_gh_2)

In [77]:
df_at = to_astype_test(df_gh_2)

In [91]:
    with open('/Users/maxwell.lee/OneDrive - Jet/New Folder/Queries/date_columns.txt') as f:
        date_columns = f.read().splitlines()

In [95]:
df_gh_v2[date_columns] = pd.to_datetime(df_gh_v2[date_columns], errors = 'coerce')

ValueError: to assemble mappings requires at least that [year, month, day] be specified: [day,month,year] is missing